# Part I

### Formulation

| | | |
| --- | --- | --- |
| Data      |   |                                                                     |
| $I$      | = | set of factories indexed by $i$ = {Cleveland, Austin}               |
| $J$      | = | set of wholesalers indexed by $j$ = {1, 2, 3, 4, 5, 6}              |
| $k_i$    | = | capacity of factory $i$                                             |
| $d_j$    | = | requirement of wholesaler $j$                                       |
| $c_{ij}$ | = | cost of distribution for one ton from factory $i$ to wholesaler $j$ |
| Let      |   |                                                                     |
| $x_{ij}$ | = | number of tons distributed from factory $i$ to wholesaler $j$       |

$\displaystyle \min. \sum_{i\in I}\sum_{j\in J} c_{ij}x_{ij}$

| | | | | | |
| --- | --- | --- | --- | --- | --- |
| s.t. | $\displaystyle\sum_{j\in J} x_{ij}$ | $\le$ | $k_i$ | $\forall i\in I$         | {factory capacity}       |
| | $\displaystyle\sum_{i\in I} x_{ij}$ | $\ge$ | $d_j$ | $\forall j\in J$         | {wholesaler requirement} |
| | $x_{ij}$                            | $\ge$ | $0$   | $\forall i\in I, j\in J$ | {non-negativity}         |

### Gurobi Model

In [1]:
# import packages
import gurobipy as gp
from gurobipy import GRB
import sensitivity_analysis as sa

In [2]:
# Set up the data
factories = ['Cleveland', 'Austin']
wholesalers = ['W1', 'W2', 'W3', 'W4', 'W5', 'W6']

capacity = {'Cleveland': 150000,
            'Austin': 200000}
requirement = {'W1': 50000,
          'W2': 10000,
          'W3': 40000,
          'W4': 35000,
          'W5': 60000,
          'W6': 20000}

costs = {
    'Cleveland': [1.00, 1.25, 1.50, 2.00, 3.00, 1.00],
    'Austin': [2.00, 1.50, 1.25, 1.75, 3.00, 1.25]}

In [3]:
# Create the model
d_1 = gp.Model('Distribution_Part_I')
d_1.ModelSense = GRB.MINIMIZE

Restricted license - for non-production use only - expires 2026-11-23


In [4]:
# Create decision variables
dvars = d_1.addVars(factories, wholesalers, lb=0.0, vtype=GRB.CONTINUOUS, name='x')

d_1.update()
dvars

{('Cleveland', 'W1'): <gurobi.Var x[Cleveland,W1]>,
 ('Cleveland', 'W2'): <gurobi.Var x[Cleveland,W2]>,
 ('Cleveland', 'W3'): <gurobi.Var x[Cleveland,W3]>,
 ('Cleveland', 'W4'): <gurobi.Var x[Cleveland,W4]>,
 ('Cleveland', 'W5'): <gurobi.Var x[Cleveland,W5]>,
 ('Cleveland', 'W6'): <gurobi.Var x[Cleveland,W6]>,
 ('Austin', 'W1'): <gurobi.Var x[Austin,W1]>,
 ('Austin', 'W2'): <gurobi.Var x[Austin,W2]>,
 ('Austin', 'W3'): <gurobi.Var x[Austin,W3]>,
 ('Austin', 'W4'): <gurobi.Var x[Austin,W4]>,
 ('Austin', 'W5'): <gurobi.Var x[Austin,W5]>,
 ('Austin', 'W6'): <gurobi.Var x[Austin,W6]>}

In [5]:
# Set the objective funtion
d_1.setObjective(gp.quicksum(costs[factory][j]*dvars[(factory, wholesaler)]
                              for factory in factories
                              for j, wholesaler in enumerate(wholesalers)))

d_1.update()
d_1.display()

Minimize
x[Cleveland,W1] + 1.25 x[Cleveland,W2] + 1.5 x[Cleveland,W3] + 2.0 x[Cleveland,W4]
+ 3.0 x[Cleveland,W5] + x[Cleveland,W6] + 2.0 x[Austin,W1] + 1.5 x[Austin,W2]
+ 1.25 x[Austin,W3] + 1.75 x[Austin,W4] + 3.0 x[Austin,W5] + 1.25 x[Austin,W6]
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\4138696333.py:7: DeprecationWarning: Model.display() is deprecated
  d_1.display()


In [6]:
d_1.printStats()

Statistics for model 'Distribution_Part_I':
  Problem type                : LP
  Linear constraint matrix    : 0 rows, 12 columns, 0 nonzeros
  Variable types              : 12 continuous, 0 integer (0 binary)
  Matrix range                : [0e+00, 0e+00]
  Objective range             : [1e+00, 3e+00]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [0e+00, 0e+00]


In [7]:
# Adding the constraints

# Factory capacity
for factory in factories:
    d_1.addLConstr(gp.quicksum(dvars[(factory, wholesaler)] for wholesaler in wholesalers),
                    GRB.LESS_EQUAL,
                    capacity[factory],
                    name=f'capacity_{factory}')

# Wholesaler requiurement
for wholesaler in wholesalers:
    d_1.addLConstr(gp.quicksum(dvars[(factory, wholesaler)] for factory in factories),
                    GRB.GREATER_EQUAL,
                    requirement[wholesaler],
                    name=f'requirement_{wholesaler}')

d_1.update()
d_1.display()

Minimize
x[Cleveland,W1] + 1.25 x[Cleveland,W2] + 1.5 x[Cleveland,W3] + 2.0 x[Cleveland,W4]
+ 3.0 x[Cleveland,W5] + x[Cleveland,W6] + 2.0 x[Austin,W1] + 1.5 x[Austin,W2]
+ 1.25 x[Austin,W3] + 1.75 x[Austin,W4] + 3.0 x[Austin,W5] + 1.25 x[Austin,W6]
Subject To
capacity_Cleveland: x[Cleveland,W1] + x[Cleveland,W2] + x[Cleveland,W3] +
 x[Cleveland,W4] + x[Cleveland,W5] + x[Cleveland,W6] <= 150000
capacity_Austin: x[Austin,W1] + x[Austin,W2] + x[Austin,W3] + x[Austin,W4] +
 x[Austin,W5] + x[Austin,W6] <= 200000
  requirement_W1: x[Cleveland,W1] + x[Austin,W1] >= 50000
  requirement_W2: x[Cleveland,W2] + x[Austin,W2] >= 10000
  requirement_W3: x[Cleveland,W3] + x[Austin,W3] >= 40000
  requirement_W4: x[Cleveland,W4] + x[Austin,W4] >= 35000
  requirement_W5: x[Cleveland,W5] + x[Austin,W5] >= 60000
  requirement_W6: x[Cleveland,W6] + x[Austin,W6] >= 20000


C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\345665737.py:18: DeprecationWarning: Model.display() is deprecated
  d_1.display()


In [8]:
# Optimize
d_1.optimize()

# Getting the results out
print(f'To have the lowest cost of ${d_1.ObjVal:0.2f}, you should ship the following amounts:')
for v in d_1.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x7200a834
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.150000e+05   0.000000e+00      0s
       6    3.7375000e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.737500000e+05
To have the lowest cost of $373750.00, you should ship the following amounts:
   x[Cleveland,W1] = 50000.0
   x[Cleveland,W2] = 10000.0
   x[Cleve

In [9]:
sa.sa_vars(d_1.getVars())

,final_value,reduced_cost,obj_coef,range_opt_low,range_opt_high
"x[Cleveland,W1]",50000.0,0.00,1.00,0.00,2.00
"x[Cleveland,W2]",10000.0,0.00,1.25,0.00,1.50
"x[Cleveland,W3]",0.0,0.25,1.50,1.25,inf
"x[Cleveland,W4]",0.0,0.25,2.00,1.75,inf
"x[Cleveland,W5]",60000.0,0.00,3.00,0.00,3.00
"x[Cleveland,W6]",20000.0,0.00,1.00,0.00,1.25
"x[Austin,W1]",0.0,1.00,2.00,1.00,inf
"x[Austin,W2]",0.0,0.25,1.50,1.25,inf
"x[Austin,W3]",40000.0,0.00,1.25,0.00,1.50
"x[Austin,W4]",35000.0,0.00,1.75,0.00,2.00


In [10]:
sa.sa_constrs(d_1.getConstrs())

,binding?,final_value,RHS,slack,shadow_price,range_feas_low,range_feas_high
capacity_Cleveland,non-binding,140000.0,150000.0,10000.0,0.00,140000.0,inf
capacity_Austin,non-binding,75000.0,200000.0,125000.0,0.00,75000.0,inf
requirement_W1,binding,50000.0,50000.0,0.0,1.00,-0.0,60000.0
requirement_W2,binding,10000.0,10000.0,0.0,1.25,-0.0,20000.0
requirement_W3,binding,40000.0,40000.0,0.0,1.25,-0.0,165000.0
requirement_W4,binding,35000.0,35000.0,0.0,1.75,-0.0,160000.0
requirement_W5,binding,60000.0,60000.0,0.0,3.00,-0.0,70000.0
requirement_W6,binding,20000.0,20000.0,0.0,1.00,-0.0,30000.0


# Part II

### Formulation

|          |   |                                                                                        |
| -------- | - | -------------------------------------------------------------------------------------- |
| Data     |   |                                                                                        |
| $I$      | = | set of factories indexed by $i$ = {Cleveland, Austin}                                  |
| $Q$      | = | set of distribution centers indexed by $q$ = {Pittsburgh, Boulder, Des Moines, Tucson} |
| $J$      | = | set of wholesalers indexed by $j$ = {1, 2, 3, 4, 5, 6}                                 |
| $k_i$    | = | capacity of factory $i$                                                                |
| $t_q$    | = | throughput capacity of distribution center $q$                                         |
| $d_j$    | = | requirement of wholesaler $j$                                                          |
| $c_{ij}$ | = | cost per ton from factory $i$ to wholesaler $j$                                        |
| $c_{iq}$ | = | cost per ton from factory $i$ to distribution center $q$                               |
| $c_{qj}$ | = | cost per ton from distribution center $q$ to wholesaler $j$                            |
| Let      |   |                                                                                        |
| $x_{ij}$ | = | tons shipped from factory $i$ to wholesaler $j$                                        |
| $x_{iq}$ | = | tons shipped from factory $i$ to distribution center $q$                               |
| $x_{qj}$ | = | tons shipped from distribution center $q$ to wholesaler $j$                            |

$\displaystyle \min. \sum_{i\in I}\sum_{j\in J} c_{ij}x_{ij} + \sum_{i\in I}\sum_{q\in Q} c_{iq}x_{iq} + \sum_{q\in Q}\sum_{j\in J} c_{qj}x_{qj}$

| | | | | | | | |
| ---- | --- | --- | --- | --- | --- | --- | --- |
| s.t. | $\displaystyle\sum_{q\in Q}x_{iq}$ | $+$ | $\displaystyle\sum_{j\in J}x_{ij}$ | $\le$ | $k_i$  | $\forall i\in I$  | {factory capacity}                     |
|   |  | | $\displaystyle\sum_{i\in I}x_{iq}$   | $=$  | $\displaystyle\sum_{j\in J}x_{qj}$ | $\forall q\in Q$  | {distribution flow}     |
|  |  |  | $\displaystyle\sum_{j\in J}x_{qj}$                       | $\le$ | $t_q$ | $\forall q\in Q$                 | {dc throughput} |
|      | $\displaystyle\sum_{i\in I}x_{ij}$ | $+$ | $\displaystyle\sum_{q\in Q}x_{qj}$ | $\ge$ | $d_j$    | $\forall j\in J$                 | {wholesaler requirement}               |
|   | |  | $x_{ij}$, $  x_{iq}$, $  x_{qj}$   | $\ge$ | $0$   | $\forall i\in I$, $  q\in Q$, $  j\in J$ | {non-negativity}     |

### Gurobi Model

In [11]:
# Set up the data
factories = ['Cleveland', 'Austin']
wholesalers = ['W1', 'W2', 'W3', 'W4', 'W5', 'W6']
dcenters = ['Pittsburgh', 'Boulder', 'Des Moines', 'Tucson']

capacity = {'Cleveland': 150000,
            'Austin': 200000}
requirement = {'W1': 50000,
          'W2': 10000,
          'W3': 40000,
          'W4': 35000,
          'W5': 60000,
          'W6': 20000}
throughput = {'Pittsburgh': 70000,
          'Boulder': 50000,
          'Des Moines': 100000,
          'Tucson': 40000}

# Costs for factory to wholesaler
fwcosts = {
    'Cleveland': {'W1': 1.00, 'W3': 1.50, 'W4': 2.00, 'W6': 1.00},
    'Austin': {'W1': 2.00}}

# Costs for factory to distribution center
fdcosts = {
    'Cleveland': {'Pittsburgh': 0.50, 'Boulder': 0.50, 'Des Moines': 1.00, 'Tucson': 0.20},
    'Austin': {'Boulder': 0.30, 'Des Moines': 0.50, 'Tucson': 0.20}}

# Costs for distribution center to wholesaler
dwcosts = {
    'Pittsburgh': {'W2': 1.50, 'W3': 0.50, 'W4': 1.50, 'W6': 1.00},
    'Boulder': {'W1': 1.00, 'W2': 0.50, 'W3': 0.50, 'W4': 1.00, 'W5': 0.50},
    'Des Moines': {'W2': 1.50, 'W3': 2.00, 'W5': 0.50, 'W6': 1.50},
    'Tucson': {'W3': 0.20, 'W4': 1.50, 'W5': 0.50, 'W6': 1.50}}

In [12]:
# Create the model
d_2 = gp.Model('Distribution_Part_II')

In [13]:
# Create decision variables
xij = d_2.addVars(
    ((i, j) for i in fwcosts for j in fwcosts[i]),
    name='xij', lb=0
)

xiq = d_2.addVars(
    ((i, q) for i in fdcosts for q in fdcosts[i]),
    name='xiq', lb=0
)

xqj = d_2.addVars(
    ((q, j) for q in dwcosts for j in dwcosts[q]),
    name='xqj', lb=0
)

d_2.update()

In [14]:
# Setting the objective
d_2.setObjective(
    gp.quicksum(fwcosts[i][j] * xij[i, j] for i in fwcosts for j in fwcosts[i]) +
    gp.quicksum(fdcosts[i][q] * xiq[i, q] for i in fdcosts for q in fdcosts[i]) +
    gp.quicksum(dwcosts[q][j] * xqj[q, j] for q in dwcosts for j in dwcosts[q]),
    GRB.MINIMIZE)

d_2.update()
d_2.display()

Minimize
xij[Cleveland,W1] + 1.5 xij[Cleveland,W3] + 2.0 xij[Cleveland,W4] + xij[Cleveland,W6]
+ 2.0 xij[Austin,W1] + 0.5 xiq[Cleveland,Pittsburgh] + 0.5 xiq[Cleveland,Boulder]
+ xiq[Cleveland,Des Moines] + 0.2 xiq[Cleveland,Tucson] + 0.3 xiq[Austin,Boulder]
+ 0.5 xiq[Austin,Des Moines] + 0.2 xiq[Austin,Tucson] + 1.5 xqj[Pittsburgh,W2]
+ 0.5 xqj[Pittsburgh,W3] + 1.5 xqj[Pittsburgh,W4] + xqj[Pittsburgh,W6] + xqj[Boulder,W1]
+ 0.5 xqj[Boulder,W2] + 0.5 xqj[Boulder,W3] + xqj[Boulder,W4] + 0.5 xqj[Boulder,W5]
+ 1.5 xqj[Des Moines,W2] + 2.0 xqj[Des Moines,W3] + 0.5 xqj[Des Moines,W5]
+ 1.5 xqj[Des Moines,W6] + 0.2 xqj[Tucson,W3] + 1.5 xqj[Tucson,W4] + 0.5 xqj[Tucson,W5]
+ 1.5 xqj[Tucson,W6]
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\3701214021.py:9: DeprecationWarning: Model.display() is deprecated
  d_2.display()


In [15]:
d_2.printStats()

Statistics for model 'Distribution_Part_II':
  Problem type                : LP
  Linear constraint matrix    : 0 rows, 29 columns, 0 nonzeros
  Variable types              : 29 continuous, 0 integer (0 binary)
  Matrix range                : [0e+00, 0e+00]
  Objective range             : [2e-01, 2e+00]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [0e+00, 0e+00]


In [16]:
# Adding the constraints

# Factory capacity
for i in factories:
    outflow_fw = gp.quicksum(xij[i, j] for j in wholesalers if (i, j) in xij)
    outflow_fd = gp.quicksum(xiq[i, q] for q in dcenters if (i, q) in xiq)
    d_2.addConstr(outflow_fw + outflow_fd <= capacity[i], name=f'capacity_{i}')

# Wholesaler requirement
for j in wholesalers:
    inflow_fw = gp.quicksum(xij[i, j] for i in factories if (i, j) in xij)
    inflow_dw = gp.quicksum(xqj[q, j] for q in dcenters if (q, j) in xqj)
    d_2.addConstr(inflow_fw + inflow_dw == requirement[j], name=f'requirement_{j}')

# DC throughput
for q in dcenters:
    d_2.addConstr(
        gp.quicksum(xqj[q, j] for j in wholesalers if (q, j) in xqj)
        <= throughput[q], name=f'throughput_{q}'
    )

# Distribution flow
for q in dcenters:
    inflow = gp.quicksum(xiq[i, q] for i in factories if (i, q) in xiq)
    outflow = gp.quicksum(xqj[q, j] for j in wholesalers if (q, j) in xqj)
    d_2.addConstr(inflow == outflow, name=f'flow_{q}')

d_2.update()
d_2.display()

Minimize
xij[Cleveland,W1] + 1.5 xij[Cleveland,W3] + 2.0 xij[Cleveland,W4] + xij[Cleveland,W6]
+ 2.0 xij[Austin,W1] + 0.5 xiq[Cleveland,Pittsburgh] + 0.5 xiq[Cleveland,Boulder]
+ xiq[Cleveland,Des Moines] + 0.2 xiq[Cleveland,Tucson] + 0.3 xiq[Austin,Boulder]
+ 0.5 xiq[Austin,Des Moines] + 0.2 xiq[Austin,Tucson] + 1.5 xqj[Pittsburgh,W2]
+ 0.5 xqj[Pittsburgh,W3] + 1.5 xqj[Pittsburgh,W4] + xqj[Pittsburgh,W6] + xqj[Boulder,W1]
+ 0.5 xqj[Boulder,W2] + 0.5 xqj[Boulder,W3] + xqj[Boulder,W4] + 0.5 xqj[Boulder,W5]
+ 1.5 xqj[Des Moines,W2] + 2.0 xqj[Des Moines,W3] + 0.5 xqj[Des Moines,W5]
+ 1.5 xqj[Des Moines,W6] + 0.2 xqj[Tucson,W3] + 1.5 xqj[Tucson,W4] + 0.5 xqj[Tucson,W5]
+ 1.5 xqj[Tucson,W6]
Subject To
capacity_Cleveland: xij[Cleveland,W1] + xij[Cleveland,W3] + xij[Cleveland,W4] +
xij[Cleveland,W6] + xiq[Cleveland,Pittsburgh] + xiq[Cleveland,Boulder] +
 xiq[Cleveland,Des Moines] + xiq[Cleveland,Tucson] <= 150000
capacity_Austin: xij[Austin,W1] + xiq[Austin,Boulder] + xiq[Austin,Des Moines] +

C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\4292727525.py:29: DeprecationWarning: Model.display() is deprecated
  d_2.display()


In [17]:
# Optimize
d_2.optimize()

# Getting the results out
print(f'To have the lowest cost of ${d_2.ObjVal:0.2f}, you should ship the following amounts:')
for v in d_2.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0xda11992c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   2.624375e+04   0.000000e+00      0s
       4    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.985000000e+05
To have the lowest cost of $198500.00, you should ship the following amounts:
   xij[Cleveland,W1] = 50000

In [18]:
sa.sa_vars(d_2.getVars())

,final_value,reduced_cost,obj_coef,range_opt_low,range_opt_high
"xij[Cleveland,W1]",50000.0,0.0,1.0,-inf,1.5
"xij[Cleveland,W3]",0.0,0.8,1.5,0.7,inf
"xij[Cleveland,W4]",0.0,0.5,2.0,1.5,inf
"xij[Cleveland,W6]",20000.0,0.0,1.0,-inf,1.5
"xij[Austin,W1]",0.0,1.0,2.0,1.0,inf
"xiq[Cleveland,Pittsburgh]",0.0,0.0,0.5,0.2,inf
"xiq[Cleveland,Boulder]",0.0,0.2,0.5,0.3,inf
"xiq[Cleveland,Des Moines]",0.0,0.5,1.0,0.5,inf
"xiq[Cleveland,Tucson]",40000.0,0.0,0.2,-inf,0.2
"xiq[Austin,Boulder]",50000.0,0.0,0.3,-inf,0.5


In [19]:
sa.sa_constrs(d_2.getConstrs())

,binding?,final_value,RHS,slack,shadow_price,range_feas_low,range_feas_high
capacity_Cleveland,non-binding,110000.0,150000.0,40000.0,0.0,110000.0,inf
capacity_Austin,non-binding,105000.0,200000.0,95000.0,0.0,105000.0,inf
requirement_W1,binding,50000.0,50000.0,0.0,1.0,0.0,90000.0
requirement_W2,binding,10000.0,10000.0,0.0,1.0,0.0,15000.0
requirement_W3,binding,40000.0,40000.0,0.0,0.7,0.0,40000.0
requirement_W4,binding,35000.0,35000.0,0.0,1.5,0.0,40000.0
requirement_W5,binding,60000.0,60000.0,0.0,1.0,5000.0,105000.0
requirement_W6,binding,20000.0,20000.0,0.0,1.0,0.0,60000.0
throughput_Pittsburgh,non-binding,0.0,70000.0,70000.0,0.0,0.0,inf
throughput_Boulder,binding,50000.0,50000.0,0.0,-0.2,45000.0,105000.0


# Part III

In [20]:
# Create the model
d_3 = gp.Model('Distribution_Part_III')

In [21]:
# Create decision variables
xij = d_3.addVars(
    ((i, j) for i in fwcosts for j in fwcosts[i]),
    name='xij', lb=0
)

xiq = d_3.addVars(
    ((i, q) for i in fdcosts for q in fdcosts[i]),
    name='xiq', lb=0
)

xqj = d_3.addVars(
    ((q, j) for q in dwcosts for j in dwcosts[q]),
    name='xqj', lb=0
)

d_3.update()

In [22]:
# Setting the objective
d_3.setObjective(
    gp.quicksum(fwcosts[i][j] * xij[i, j] for i in fwcosts for j in fwcosts[i]) +
    gp.quicksum(fdcosts[i][q] * xiq[i, q] for i in fdcosts for q in fdcosts[i]) +
    gp.quicksum(dwcosts[q][j] * xqj[q, j] for q in dwcosts for j in dwcosts[q]),
    GRB.MINIMIZE)

d_3.update()
d_3.display()

Minimize
xij[Cleveland,W1] + 1.5 xij[Cleveland,W3] + 2.0 xij[Cleveland,W4] + xij[Cleveland,W6]
+ 2.0 xij[Austin,W1] + 0.5 xiq[Cleveland,Pittsburgh] + 0.5 xiq[Cleveland,Boulder]
+ xiq[Cleveland,Des Moines] + 0.2 xiq[Cleveland,Tucson] + 0.3 xiq[Austin,Boulder]
+ 0.5 xiq[Austin,Des Moines] + 0.2 xiq[Austin,Tucson] + 1.5 xqj[Pittsburgh,W2]
+ 0.5 xqj[Pittsburgh,W3] + 1.5 xqj[Pittsburgh,W4] + xqj[Pittsburgh,W6] + xqj[Boulder,W1]
+ 0.5 xqj[Boulder,W2] + 0.5 xqj[Boulder,W3] + xqj[Boulder,W4] + 0.5 xqj[Boulder,W5]
+ 1.5 xqj[Des Moines,W2] + 2.0 xqj[Des Moines,W3] + 0.5 xqj[Des Moines,W5]
+ 1.5 xqj[Des Moines,W6] + 0.2 xqj[Tucson,W3] + 1.5 xqj[Tucson,W4] + 0.5 xqj[Tucson,W5]
+ 1.5 xqj[Tucson,W6]
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\4146266278.py:9: DeprecationWarning: Model.display() is deprecated
  d_3.display()


In [23]:
# Adding the constraints

# Factory capacity
for i in factories:
    outflow_fw = gp.quicksum(xij[i, j] for j in wholesalers if (i, j) in xij)
    outflow_fd = gp.quicksum(xiq[i, q] for q in dcenters if (i, q) in xiq)
    d_3.addConstr(outflow_fw + outflow_fd <= capacity[i], name=f'capacity_{i}')

# Wholesaler requirement
for j in wholesalers:
    inflow_fw = gp.quicksum(xij[i, j] for i in factories if (i, j) in xij)
    inflow_dw = gp.quicksum(xqj[q, j] for q in dcenters if (q, j) in xqj)
    d_3.addConstr(inflow_fw + inflow_dw == requirement[j], name=f'requirement_{j}')

# DC throughput
for q in dcenters:
    d_3.addConstr(
        gp.quicksum(xqj[q, j] for j in wholesalers if (q, j) in xqj)
        <= throughput[q], name=f'throughput_{q}'
    )

# Distribution flow
for q in dcenters:
    inflow = gp.quicksum(xiq[i, q] for i in factories if (i, q) in xiq)
    outflow = gp.quicksum(xqj[q, j] for j in wholesalers if (q, j) in xqj)
    d_3.addConstr(inflow == outflow, name=f'flow_{q}')

d_3.update()
d_3.display()

Minimize
xij[Cleveland,W1] + 1.5 xij[Cleveland,W3] + 2.0 xij[Cleveland,W4] + xij[Cleveland,W6]
+ 2.0 xij[Austin,W1] + 0.5 xiq[Cleveland,Pittsburgh] + 0.5 xiq[Cleveland,Boulder]
+ xiq[Cleveland,Des Moines] + 0.2 xiq[Cleveland,Tucson] + 0.3 xiq[Austin,Boulder]
+ 0.5 xiq[Austin,Des Moines] + 0.2 xiq[Austin,Tucson] + 1.5 xqj[Pittsburgh,W2]
+ 0.5 xqj[Pittsburgh,W3] + 1.5 xqj[Pittsburgh,W4] + xqj[Pittsburgh,W6] + xqj[Boulder,W1]
+ 0.5 xqj[Boulder,W2] + 0.5 xqj[Boulder,W3] + xqj[Boulder,W4] + 0.5 xqj[Boulder,W5]
+ 1.5 xqj[Des Moines,W2] + 2.0 xqj[Des Moines,W3] + 0.5 xqj[Des Moines,W5]
+ 1.5 xqj[Des Moines,W6] + 0.2 xqj[Tucson,W3] + 1.5 xqj[Tucson,W4] + 0.5 xqj[Tucson,W5]
+ 1.5 xqj[Tucson,W6]
Subject To
capacity_Cleveland: xij[Cleveland,W1] + xij[Cleveland,W3] + xij[Cleveland,W4] +
xij[Cleveland,W6] + xiq[Cleveland,Pittsburgh] + xiq[Cleveland,Boulder] +
 xiq[Cleveland,Des Moines] + xiq[Cleveland,Tucson] <= 150000
capacity_Austin: xij[Austin,W1] + xiq[Austin,Boulder] + xiq[Austin,Des Moines] +

C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\3825820943.py:29: DeprecationWarning: Model.display() is deprecated
  d_3.display()


In [24]:
# Adding the preferences

# W1 preferred supplier: Cleveland factory
for i in factories:
    if i != 'Cleveland' and (i, 'W1') in xij:
        d_3.addConstr(xij[i, 'W1'] == 0)
for q in dcenters:
    if (q, 'W1') in xqj:
        d_3.addConstr(xqj[q, 'W1'] == 0)

# W2 preferred supplier: Pittsburgh DC
for q in dcenters:
    if q != 'Pittsburgh' and (q, 'W2') in xqj:
        d_3.addConstr(xqj[q, 'W2'] == 0)
for i in factories:
    if (i, 'W2') in xij:
        d_3.addConstr(xij[i, 'W2'] == 0)

# W5 preferred supplier: Boulder DC
for q in dcenters:
    if q != 'Boulder' and (q, 'W5') in xqj:
        d_3.addConstr(xqj[q, 'W5'] <= 10000) # 10,000 overflow
if ('Boulder', 'W5') in xqj:
    d_3.addConstr(xqj['Boulder', 'W5'] == 50000) # Boulder maximmum throughput is 50,000
for i in factories:
    if (i, 'W5') in xij:
        d_3.addConstr(xij[i, 'W5'] == 0)

# W6 preferred supplier: Des Moines or Tucson DC
for q in dcenters:
    if q not in ['Des Moines', 'Tucson'] and (q, 'W6') in xqj:
        d_3.addConstr(xqj[q, 'W6'] == 0)
for i in factories:
    if (i, 'W6') in xij:
        d_3.addConstr(xij[i, 'W6'] == 0)

d_3.update()
d_3.display()

Minimize
xij[Cleveland,W1] + 1.5 xij[Cleveland,W3] + 2.0 xij[Cleveland,W4] + xij[Cleveland,W6]
+ 2.0 xij[Austin,W1] + 0.5 xiq[Cleveland,Pittsburgh] + 0.5 xiq[Cleveland,Boulder]
+ xiq[Cleveland,Des Moines] + 0.2 xiq[Cleveland,Tucson] + 0.3 xiq[Austin,Boulder]
+ 0.5 xiq[Austin,Des Moines] + 0.2 xiq[Austin,Tucson] + 1.5 xqj[Pittsburgh,W2]
+ 0.5 xqj[Pittsburgh,W3] + 1.5 xqj[Pittsburgh,W4] + xqj[Pittsburgh,W6] + xqj[Boulder,W1]
+ 0.5 xqj[Boulder,W2] + 0.5 xqj[Boulder,W3] + xqj[Boulder,W4] + 0.5 xqj[Boulder,W5]
+ 1.5 xqj[Des Moines,W2] + 2.0 xqj[Des Moines,W3] + 0.5 xqj[Des Moines,W5]
+ 1.5 xqj[Des Moines,W6] + 0.2 xqj[Tucson,W3] + 1.5 xqj[Tucson,W4] + 0.5 xqj[Tucson,W5]
+ 1.5 xqj[Tucson,W6]
Subject To
capacity_Cleveland: xij[Cleveland,W1] + xij[Cleveland,W3] + xij[Cleveland,W4] +
xij[Cleveland,W6] + xiq[Cleveland,Pittsburgh] + xiq[Cleveland,Boulder] +
 xiq[Cleveland,Des Moines] + xiq[Cleveland,Tucson] <= 150000
capacity_Austin: xij[Austin,W1] + xiq[Austin,Boulder] + xiq[Austin,Des Moines] +

C:\Users\zungz\AppData\Local\Temp\ipykernel_31132\683789112.py:38: DeprecationWarning: Model.display() is deprecated
  d_3.display()


flow_Tucson: xiq[Cleveland,Tucson] + xiq[Austin,Tucson] + -1.0 xqj[Tucson,W3] + -1.0
 xqj[Tucson,W4] + -1.0 xqj[Tucson,W5] + -1.0 xqj[Tucson,W6] = 0
  R16: xij[Austin,W1] = 0
  R17: xqj[Boulder,W1] = 0
  R18: xqj[Boulder,W2] = 0
  R19: xqj[Des Moines,W2] = 0
  R20: xqj[Des Moines,W5] <= 10000
  R21: xqj[Tucson,W5] <= 10000
  R22: xqj[Boulder,W5] = 50000
  R23: xqj[Pittsburgh,W6] = 0
  R24: xij[Cleveland,W6] = 0


In [25]:
# Optimize
d_3.optimize()

# Getting the results out
print(f'To have the lowest cost of ${d_3.ObjVal:0.2f} while meeting all preferences, you should ship the following amounts:')
for v in d_3.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 25 rows, 29 columns and 84 nonzeros
Model fingerprint: 0x8b12f889
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 17 rows and 16 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0550000e+05   2.125000e+04   0.000000e+00      0s
       5    2.4600000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.460000000e+05
To have the lowest cost of $246000.00 while meeting all preferences, you should ship the following amount

In [26]:
# Optimize
d_3.optimize()

# Getting the results out
print(f'To have the lowest cost of ${d_3.ObjVal:0.2f}, you should ship the following amounts:')
for v in d_3.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 25 rows, 29 columns and 84 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.460000000e+05
To have the lowest cost of $246000.00, you should ship the following amounts:
   xij[Cleveland,W1] = 50000.0
   xij[Cleveland,W3] = 0.0
   xij[Cleveland,W4] = 0.0
   xij[Cleveland,W6] = 0.0
   xij[Austin,W1] = 0.0
   xiq[Cleveland,Pittsburgh] = 45000.0
   xiq[Cleveland,Boulder] = 0.0
   xiq[Cleveland,Des Moines] = 0.0
   xiq[Cleveland,Tucson] = 40000.0
   xiq[Austin,Boulder] = 50000.0
   xiq[Austin,Des Moines] = 30000.0
   xiq[A

In [27]:
sa.sa_vars(d_3.getVars())

,final_value,reduced_cost,obj_coef,range_opt_low,range_opt_high
"xij[Cleveland,W1]",50000.0,0.0,1.0,-inf,2.0
"xij[Cleveland,W3]",0.0,0.8,1.5,0.7,inf
"xij[Cleveland,W4]",0.0,0.0,2.0,2.0,inf
"xij[Cleveland,W6]",0.0,0.0,1.0,-inf,inf
"xij[Austin,W1]",0.0,1.0,2.0,1.0,inf
"xiq[Cleveland,Pittsburgh]",45000.0,0.0,0.5,0.2,0.5
"xiq[Cleveland,Boulder]",0.0,0.2,0.5,0.3,inf
"xiq[Cleveland,Des Moines]",0.0,0.5,1.0,0.5,inf
"xiq[Cleveland,Tucson]",40000.0,0.0,0.2,-inf,0.2
"xiq[Austin,Boulder]",50000.0,0.0,0.3,-inf,0.5


In [28]:
sa.sa_constrs(d_3.getConstrs())

,binding?,final_value,RHS,slack,shadow_price,range_feas_low,range_feas_high
capacity_Cleveland,non-binding,135000.0,150000.0,15000.0,0.0,135000.0,inf
capacity_Austin,non-binding,80000.0,200000.0,120000.0,0.0,80000.0,inf
requirement_W1,binding,50000.0,50000.0,0.0,1.0,0.0,65000.0
requirement_W2,binding,10000.0,10000.0,0.0,2.0,0.0,25000.0
requirement_W3,binding,40000.0,40000.0,0.0,0.7,20000.0,40000.0
requirement_W4,binding,35000.0,35000.0,0.0,2.0,0.0,50000.0
requirement_W5,binding,60000.0,60000.0,0.0,1.0,50000.0,60000.0
requirement_W6,binding,20000.0,20000.0,0.0,2.0,0.0,90000.0
throughput_Pittsburgh,non-binding,45000.0,70000.0,25000.0,0.0,45000.0,inf
throughput_Boulder,binding,50000.0,50000.0,0.0,-0.7,50000.0,85000.0
